In [4]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from reservoirpy.nodes import Reservoir, Ridge
from sklearn.preprocessing import StandardScaler
from collections import Counter
from numpy.linalg import inv

import CPM1
import importlib
import eval_signal1
import MA
import Stochastic1
importlib.reload(MA)
importlib.reload(eval_signal1)
importlib.reload(CPM1)
importlib.reload(Stochastic1)
from MA import MovingAverageGA, MovingAverage
from CPM import extract_cpm_points
from CPM1 import cpm_model
from Stochastic1 import run_stochastic_ga_optimization, generate_stochastic_signals

ModuleNotFoundError: No module named 'CPM1'

In [ ]:
name = 'MSFT' #MicroSoft
ticker = yf.Ticker(name)
ori_df = ticker.history(interval='1d', period='10y', auto_adjust=False)

In [ ]:
ori_df.isnull().sum()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(ori_df.index, ori_df['Close'], label='Close Price', color='blue')
plt.title(f'{name} - Close Price (10y)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df = ori_df.copy()

In [ ]:
df['MA20'] = ori_df['Close'].rolling(window=20).mean()
df['MA60'] = ori_df['Close'].rolling(window=60).mean()

In [ ]:
def detect_cross_signals(df: pd.DataFrame) -> None:
    ma_short = df['MA20']
    ma_long = df['MA60']
    
    # 이동평균의 이전 값
    prev_short = ma_short.shift(1)
    prev_long = ma_long.shift(1)

    # 골든 크로스: 단기선이 장기선을 아래에서 위로 돌파
    df['Golden'] = ((prev_short < prev_long) & (ma_short >= ma_long)).astype(int)

    # 데드 크로스: 단기선이 장기선을 위에서 아래로 돌파
    df['Dead'] = ((prev_short > prev_long) & (ma_short <= ma_long)).astype(int)
detect_cross_signals(df)

In [ ]:
plt.figure(figsize=(10, 5))

# 가격 및 이동평균선
plt.plot(df['Close'], label='Close Price', color='blue')
plt.plot(df['MA20'], label='20-day MA', color='orange')
plt.plot(df['MA60'], label='60-day MA', color='green')

plt.scatter(df.index[df['Golden'] == 1], df['Close'][df['Golden'] == 1], 
            marker='^', color='green', label='Golden Cross', s=100)

plt.scatter(df.index[df['Dead'] == 1], df['Close'][df['Dead'] == 1], 
            marker='v', color='red', label='Dead Cross', s=100)

plt.title(f'{name} - Close Price & Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df.head()

In [ ]:
cpm_points, df_with_cpm = cpm_model(df, column='Close',P=0.09, T=5)

In [5]:
df_with_cpm.head()

NameError: name 'df_with_cpm' is not defined

In [ ]:
len(cpm_points)

In [ ]:
df_with_cpm['cpm_point_type'].describe()

In [ ]:
# CPM 신호가 있는 인덱스와 가격
cpm_indices = df_with_cpm[df_with_cpm['is_cpm_point'] == True].index
cpm_prices = df_with_cpm.loc[cpm_indices, 'Close']

# Buy (1) / Sell (-1) 인덱스와 가격
buy_indices = df_with_cpm[df_with_cpm['cpm_point_type'] == 1].index
sell_indices = df_with_cpm[df_with_cpm['cpm_point_type'] == -1].index
buy_prices = df_with_cpm.loc[buy_indices, 'Close']
sell_prices = df_with_cpm.loc[sell_indices, 'Close']

plt.figure(figsize=(15, 6))

# 1. 전체 가격 라인
plt.plot(df.index, df_with_cpm['Close'], label='Price', color='blue')

# 2. CPM 전환점을 선으로 연결
plt.plot(cpm_indices, cpm_prices, label='CPM Turning Points (line)', color='orange')

# 3. Buy 시점 (▲ 초록색)
#plt.scatter(buy_indices, buy_prices, label='Buy Signal', marker='^', color='green', s=100)

# 4. Sell 시점 (▼ 빨간색)
#plt.scatter(sell_indices, sell_prices, label='Sell Signal', marker='v', color='red', s=100)

plt.title("Price with CPM Turning Points, Buy/Sell Signals")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_len = len(df)

# # 6:2:2
# train_end = int(total_len * 0.6)
# val_end = int(total_len * 0.8)

# train_df = df.iloc[:train_end]
# val_df = df.iloc[train_end:val_end]
# test_df = df.iloc[val_end:]

train_end = int(total_len * 0.7)
train_df = df_with_cpm.iloc[:train_end]
test_df = df_with_cpm.iloc[train_end:]

In [ ]:
st_best_params, st_best_fitness, train_df = run_stochastic_ga_optimization(train_df)

print("Best Parameters:", st_best_params)
print("Best Fitness:", st_best_fitness)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_raw = np.concatenate([
    train_df['ST_Signals'].values.reshape(-1, 1),
    train_df['Close'].values.reshape(-1, 1)
], axis=1)
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)

Y_train = train_df['cpm_point_type'].values.reshape(-1, 1)

In [ ]:
test_df_result = test_df.copy()

st_period, st_a, st_b, st_c, st_d = st_best_params
st_period = int(st_period)

suggested_signals_from_best_params = generate_stochastic_signals(test_df_result, st_period, st_a, st_b, st_c, st_d)

test_df_result['ST_Signals'] = 0

if not suggested_signals_from_best_params.empty:
    signal_map = suggested_signals_from_best_params.set_index('Index')['Type'].map({'BUY': -1, 'SELL': 1})
    test_df_result['ST_Signals'] = test_df_result['ST_Signals'].combine_first(signal_map).fillna(0).astype(int)

test_df = test_df_result

In [ ]:
X_test_raw = np.concatenate([
    test_df['ST_Signals'].values.reshape(-1, 1),
    test_df['Close'].values.reshape(-1, 1)
], axis=1)
X_test = scaler.transform(X_test_raw)

Y_test = test_df['cpm_point_type'].values.reshape(-1, 1)

In [ ]:
# 모델 정의
reservoir = Reservoir(units=1000, input_dim=2, lr=0.2, sr=0.9)
ridge = Ridge(ridge=1)
esn_model = reservoir >> ridge

# 학습
X_train = X_train
esn_model.fit(X_train, Y_train)

In [ ]:
Y_pred = esn_model.run(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Y_pred가 연속된 값일 수 있으므로, 이진 분류로 변환
# 예: 0.5 이상이면 1, -0.5 이하이면 -1, 나머지는 0
def convert_signals(y):
    return np.where(y > 0.05, 1, np.where(y < -0.01, -1, 0))

Y_pred_class = convert_signals(Y_pred)
Y_test_class = convert_signals(Y_test)

# confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test_class, Y_pred_class, labels=[1, 0, -1]))

# classification report (precision, recall, f1)
print("\nClassification Report:")
print(classification_report(Y_test_class, Y_pred_class, labels=[1, 0, -1], digits=3))


In [ ]:
from pyESN import ESN

In [ ]:
pyesn_model = ESN(
    n_inputs=2,  # 입력 특징의 수
    n_outputs=1,        # 출력 (타겟)의 수
    n_reservoir=500,                   # 리저버 뉴런 수
    spectral_radius=0.95,              # 스펙트럼 반경
    sparsity=0,                  # 순환 가중치
    noise=0.001,                 # 노이즈
    input_shift=None,            # 입력 값에 더함
    input_scaling=None,          # 입력 값에 곱함
    teacher_forcing=True,        # 정답을 다음 스텝 출력에 사용할 건가?(학습은 True, 예측은 자동으로 False)
    feedback_scaling=None,       #
    teacher_scaling=None,        # 타겟값에 곱함
    teacher_shift=None,          # 타겟값에 더함
    #out_activation=np.tanh,     # 출력층에 사용할 함수(기본 identity-> 아무것도 안함), tanh, sigmoid
    #inverse_out_activation=np.arctanh, # 역함수로 출력값 복원
    random_state=None,           # 무작위성 제어
    silent=True                  # 내부 메시지 출력
)

In [ ]:
pyesn_model.fit(X_train, Y_train)

In [ ]:
prediction = pyesn_model.predict(X_test)

In [ ]:
def to_nearest_class(value):
    classes = np.array([-1, 0, 1])
    idx = np.abs(classes - value).argmin()
    return classes[idx]

Y_pred_pyesn = np.array([to_nearest_class(y) for y in prediction])

In [ ]:
# confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred_pyesn, labels=[1, 0, -1]))

# classification report (precision, recall, f1)
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred_pyesn, labels=[1, 0, -1], digits=3))

In [6]:
# CPM 신호가 있는 인덱스와 가격
cpm_indices = test_df[test_df['is_cpm_point'] == True].index
cpm_prices = test_df.loc[cpm_indices, 'Close']

# Buy (1) / Sell (-1) 인덱스와 가격
buy_indices = test_df[test_df['cpm_point_type'] == 1].index
sell_indices = test_df[test_df['cpm_point_type'] == -1].index
buy_prices = test_df.loc[buy_indices, 'Close']
sell_prices = test_df.loc[sell_indices, 'Close']

buy_indices = test_df[test_df['cpm_point_type'] == 1].index
sell_indices = test_df[test_df['cpm_point_type'] == -1].index
buy_prices = test_df.loc[buy_indices, 'Close']
sell_prices = test_df.loc[sell_indices, 'Close']

plt.figure(figsize=(15, 6))

# 1. 전체 가격 라인
plt.plot(test_df.index, test_df['Close'], label='Price', color='blue')

# 2. CPM 전환점을 선으로 연결
plt.plot(cpm_indices, cpm_prices, label='CPM Turning Points (line)', color='orange')

# 예측
plt.plot(cpm_indices, cpm_prices, label='Prediction (line)', color='green')

# 3. Buy 시점 (▲ 초록색)
#plt.scatter(buy_indices, buy_prices, label='Buy Signal', marker='^', color='green', s=100)

# 4. Sell 시점 (▼ 빨간색)
#plt.scatter(sell_indices, sell_prices, label='Sell Signal', marker='v', color='red', s=100)

plt.title("Price with CPM Turning Points, Buy/Sell Signals")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'test_df' is not defined